## Modules

In [1]:
import os
import sys
sys.path.append('..')

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from bsoption.bsmodel import BSModel
from bsoption.nyopchain import NYopchain
Opchain = NYopchain()

from litedata import *
from litedata.litedb import Litedata
from litedata.loadyahoodata import *

Litedb = Litedata('yahoo')

### US stock list setting

In [2]:
from configparser import ConfigParser
configchain = ConfigParser()
configchain.read('setting_yahoochain.ini')
chainpath = configchain['paths'].get('chainpath')

configassets = configchain['assets']
etflist = list(configassets.get('etf').split(','))
chipslist = list(configassets.get('chip').split(','))
assetlist = etflist + chipslist

## Gamma Exposure (GEX)

### OHLC of all stocks

In [3]:
dfohlcall = getyahoodata(assetlist)

[*********************100%***********************]  24 of 24 completed


d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-p

### Option chain with closing price >= 0.05 with greeks

In [4]:
def loadopchain(daystr='2022-10-03', expiry='2022-10-14', asset='NVDA', bounds=(0, 1000), opbound=0.05):
    """Obtain all options of the same expiry with closing price > 0.01 with greeks."""
    # inputdict for `.loadopdata()` method and query
    inputdict = {'asset': asset,  'optype': ('C', 'P'), 
                 'strike_lowerbound': bounds[0], 'strike_upperbound': bounds[1],  
                 'startexpiry': expiry,  'endexpiry': expiry,  'starttd': daystr,  'endtd': daystr}
    dfop = Opchain.loadopdata(inputdict)
    # Filter strike price with BOTH call & put price over 1.00
    dfcall = dfop[(dfop['optype'] == 'C') & (dfop['mid'] >= opbound)]
    dfput = dfop[(dfop['optype'] == 'P') & (dfop['mid'] >= opbound)]
    strikeset = set(dfcall['strike']).intersection(set(dfput['strike']))
    dfop = dfop[dfop['strike'].isin(strikeset)]
    # Underlying close price
    spotprice = dfohlcall.loc[daystr, f'{asset}_cl']
    dfop[f'{asset}_cl'] = spotprice
    # Dummy column of `BSmodel()` object
    tdays = (datetime.strptime(expiry, '%Y-%m-%d') - datetime.strptime(daystr, '%Y-%m-%d')).days
    dfop['BS'] = dfop.apply(lambda row: BSModel(spotprice, row['strike'], tdays, row['iv'] / 100), axis=1)
    # split option dataframe into call & put and compute delta and theta separately
    dfcall = dfop[dfop['optype'] == 'C']
    dfput = dfop[dfop['optype'] == 'P']
    dfcall['delta'] = dfcall['BS'].apply(lambda x: x.cdelta)
    dfcall['theta'] = dfcall['BS'].apply(lambda x: x.ctheta)
    dfput['delta'] = dfput['BS'].apply(lambda x: x.pdelta)
    dfput['theta'] = dfput['BS'].apply(lambda x: x.ptheta)
    dfop = pd.concat([dfcall, dfput], axis=0)
    # Compute vega & gamma columns (irrelevant of call/put)
    dfop['vega'] = dfop['BS'].apply(lambda x: x.vega)
    dfop['gamma'] = dfop['BS'].apply(lambda x: x.gamma)
    # Drop dummy column
    dfop.drop('BS', axis=1, inplace=True)
    
    return dfop

In [5]:
asset0 = 'NVDA'
dfop1 = loadopchain(daystr='2022-08-26', expiry='2022-09-16', asset=asset0, bounds=(0, 300))
dfop1

<ipython-input-4-c6e19e842c4c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcall['delta'] = dfcall['BS'].apply(lambda x: x.cdelta)
<ipython-input-4-c6e19e842c4c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcall['theta'] = dfcall['BS'].apply(lambda x: x.ctheta)
<ipython-input-4-c6e19e842c4c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid,NVDA_cl,delta,theta,vega,gamma
2,2022-08-26,NVDA,C,2022-09-16,28.75,333.20,3.0,44,159.65,131.95,136.30,134.125,162.5517,0.994873,-0.045735,0.005765,0.000114
4,2022-08-26,NVDA,C,2022-09-16,31.25,338.67,4.0,15,148.48,129.75,133.80,131.775,162.5517,0.992576,-0.064527,0.008002,0.000155
17,2022-08-26,NVDA,C,2022-09-16,47.50,254.88,5.0,411,124.65,113.50,117.55,115.525,162.5517,0.989776,-0.064295,0.010595,0.000273
42,2022-08-26,NVDA,C,2022-09-16,107.50,94.53,22.0,222,57.30,54.10,56.90,55.500,162.5517,0.973630,-0.053631,0.023828,0.001658
43,2022-08-26,NVDA,C,2022-09-16,110.00,110.64,22.0,362,55.00,51.45,53.70,52.575,162.5517,0.945665,-0.113164,0.042958,0.002554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2022-08-26,NVDA,P,2022-09-16,210.00,57.03,74.0,4407,46.85,46.45,47.70,47.075,162.5517,-0.964372,-0.041510,0.030570,0.003526
173,2022-08-26,NVDA,P,2022-09-16,215.00,78.22,111.0,2602,51.70,51.35,53.45,52.400,162.5517,-0.918741,-0.109234,0.058653,0.004932
174,2022-08-26,NVDA,P,2022-09-16,220.00,87.89,25.0,4012,52.40,55.30,58.75,57.025,162.5517,-0.908261,-0.134392,0.064222,0.004807
175,2022-08-26,NVDA,P,2022-09-16,225.00,90.99,18.0,4098,56.22,61.00,63.65,62.325,162.5517,-0.916278,-0.129956,0.059986,0.004337


### Gamma exposure function

In [6]:
def getgex(dfchain, asset='NVDA'):
    """Obtain gamma exposure of each option contract per 1% move of underlying."""
    ## Attach GEX column
    dfgex = dfchain.copy()
    putfactor = lambda x: -1 if (x == 'P') else 1
    colfactor = pd.to_numeric(((dfgex['optype']).apply(putfactor)))
    dfgex['GEX'] = dfgex['gamma'] * dfgex['oi'] * colfactor * 100
    # Split into call & put dataframe
    collist = ['tradedate', 'expiry', f'{asset}_cl', 'strike', 'mid', 'iv', 'oi', 'delta', 'gamma', 'GEX']
    dfcall = dfgex[dfgex['optype'] == 'C'][collist]
    dfcall.rename(columns={field: f'c_{field}' for field in collist[4:]}, inplace=True)
    dfcall['c_GEX'] *= dfgex[f'{asset}_cl'].iloc[0] / 100
    dfcall.set_index('strike', inplace=True)
    dfput = dfgex[dfgex['optype'] == 'P'][collist[3:]]
    dfput.rename(columns={field: f'p_{field}' for field in collist[4:]}, inplace=True)
    dfput['p_GEX'] *= dfgex[f'{asset}_cl'].iloc[0] / 100
    dfput.set_index('strike', inplace=True)
    # Concat along columns
    dfgex = pd.concat([dfcall, dfput], axis=1)
    # Net GEX (call GEX + put GEX)
    dfgex['GEX'] = dfgex['c_GEX'] + dfgex['p_GEX']
    for col in ['c_GEX', 'p_GEX', 'GEX']:
        dfgex[col] = np.round(dfgex[col], 4)
    
    return dfgex

In [7]:
dfgex1 = getgex(dfop1, asset=asset0)
dfgex1[(dfgex1['c_mid'] > 1) & (dfgex1['p_mid'] > 1)]

,tradedate,expiry,NVDA_cl,c_mid,c_iv,c_oi,c_delta,c_gamma,c_GEX,p_mid,p_iv,p_oi,p_delta,p_gamma,p_GEX,GEX
strike,,,,,,,,,,,,,,,,
140.0,2022-08-26,2022-09-16,162.5517,24.425,61.77,867,0.860399,0.009224,1299.9138,1.455,57.32,15398,-0.124038,0.009161,-22930.1435,-21630.2297
145.0,2022-08-26,2022-09-16,162.5517,19.700,54.47,856,0.826359,0.012078,1680.5210,2.270,56.16,7042,-0.179943,0.011981,-13714.6045,-12034.0835
150.0,2022-08-26,2022-09-16,162.5517,16.125,55.95,3466,0.747263,0.014651,8254.4303,3.400,54.94,17923,-0.249616,0.014823,-43184.3750,-34929.9447
155.0,2022-08-26,2022-09-16,162.5517,12.675,54.86,2521,0.665422,0.017024,6976.1329,4.900,53.63,6946,-0.332100,0.017363,-19604.2253,-12628.0923
157.5,2022-08-26,2022-09-16,162.5517,11.050,53.91,315,0.621264,0.018096,926.5764,5.850,53.24,553,-0.377873,0.018311,-1645.9691,-719.3928
160.0,2022-08-26,2022-09-16,162.5517,9.600,53.38,10004,0.574402,0.018834,30626.7047,6.850,52.41,19356,-0.425157,0.019178,-60341.3148,-29714.6101
162.5,2022-08-26,2022-09-16,162.5517,8.250,52.75,237,0.526223,0.019355,745.6469,8.000,51.78,569,-0.474222,0.019719,-1823.8502,-1078.2033
165.0,2022-08-26,2022-09-16,162.5517,7.000,51.99,9247,0.477063,0.019648,29533.1185,9.300,51.36,11458,-0.523823,0.019886,-37038.6687,-7505.5502
167.5,2022-08-26,2022-09-16,162.5517,5.925,51.59,200,0.428397,0.019513,634.3660,10.700,50.77,1990,-0.573525,0.019810,-6408.1451,-5773.7790


### Visualize GEX

In [8]:
def gettotalgex(dfgex):
    """Obtain call-GEX, put-GEX and total-GEX."""
    return round(dfgex['c_GEX'].sum(), 2), round(dfgex['p_GEX'].sum(), 2), round(dfgex['GEX'].sum(), 2)

def plotgex(dfgex, asset='NVDA'):
    """Plot call GEX, put GEX and net GEX."""
    tdate = (dfgex['tradedate'].iloc[0]).strftime('%Y-%m-%d')
    expiry = (dfgex['expiry'].iloc[0]).strftime('%Y-%m-%d')
    spotprice = dfgex[f'{asset}_cl'].iloc[0]
    gexsum_c, gexsum_p, gexsum = gettotalgex(dfgex)
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.125, row_heights=[8.0, 8.0],
                        specs=[[{"type": "bar"}], [{"type": "bar"}]], 
                        subplot_titles=(f'Call-GEX: {gexsum_c}, Put-GEX: {gexsum_p}', f'Net GEX: {gexsum}'))
        
    fig.add_trace(go.Bar(x=dfgex.index, y=dfgex['c_GEX'], name="Call GEX",
                         marker=dict(color='#34dd74', line=dict(color='#34dd74', width=2))), 
                  row=1, col=1)
    
    fig.add_trace(go.Bar(x=dfgex.index, y=dfgex['p_GEX'], name="Put GEX",
                         marker=dict(color='#dd3462', line=dict(color='#dd3462', width=2))), 
                  row=1, col=1)
    
    fig.add_trace(go.Bar(x=dfgex.index, y=dfgex['GEX'], name="GEX",
                         marker=dict(color='#1956ee', line=dict(color='#1956ee', width=2))), 
                  row=2, col=1)
    
    titletext = f'{asset} GEX of expiry {expiry} on {tdate} at {spotprice} \n'
    
    fig.update_layout(title=titletext, title_x=0.5, width=1000, height=800)
    fig.show()

In [9]:
plotgex(dfgex1, asset=asset0)

### Compute GEX for general spot price 

In [10]:
def getnewgex(dfgex, asset='NVDA', spotprice=180):
    """Obtain total GEX of an option chain with arbitary underlying price."""
    # Alter underlying price
    dfnewgex = dfgex.copy()
    dfnewgex[f'{asset}_cl'] = spotprice
    # Recompute option price, delta and gamma
    dfnewgex.reset_index(inplace=True)
    tdate = dfnewgex['tradedate'].iloc[0]
    expiry = dfnewgex['expiry'].iloc[0]
    tdays = (expiry - tdate).days
    colbs_c = dfnewgex.apply(lambda row: BSModel(spotprice, row['strike'], tdays, row['c_iv'] / 100), axis=1)
    dfnewgex['c_close'] = np.round(colbs_c.apply(lambda x: x.cprice), 2)
    dfnewgex['c_delta'] = np.round(colbs_c.apply(lambda x: x.cdelta), 4)
    dfnewgex['c_gamma'] = colbs_c.apply(lambda x: x.gamma)
    colbs_p = dfnewgex.apply(lambda row: BSModel(spotprice, row['strike'], tdays, row['p_iv'] / 100), axis=1)
    dfnewgex['p_close'] = np.round(colbs_p.apply(lambda x: x.pprice), 2)
    dfnewgex['p_delta'] = np.round(colbs_p.apply(lambda x: x.pdelta), 4)
    dfnewgex['p_gamma'] = colbs_p.apply(lambda x: x.gamma)    
    # GEX
    dfnewgex['c_GEX'] = np.round(dfnewgex['c_gamma'] * dfnewgex['c_oi'] * spotprice, 4) 
    dfnewgex['p_GEX'] = np.round(dfnewgex['p_gamma'] * dfnewgex['p_oi'] * -1 * spotprice, 4)
    dfnewgex['GEX'] = np.round(dfnewgex['c_GEX'] + dfnewgex['p_GEX'], 4)
    # Set back `strike` as index
    dfnewgex.set_index('strike', inplace=True)
    
    return dfnewgex

### Zero Gamma Level

In [11]:
def get0gamma(dfchain, asset='NVDA', spotbound=(160, 200), interval=2.5):
    """Obtain zero gamma level."""
    # GEX according to true underlying price
    dfgex = getgex(dfchain, asset)
    spotprice = dfgex[f'{asset}_cl'].mean()
    cgex0, pgex0, ngex0 = gettotalgex(dfgex)
    # Obtain GEX for all spot price levels
    spotlevels = np.arange(spotbound[0], spotbound[1], interval)
    dfsumgex = pd.DataFrame()
    for level in spotlevels:
        dfnewgex = getnewgex(dfgex, asset, level)
        cgex, pgex, ngex = gettotalgex(dfnewgex)
        dfsumgex.loc[level, 'c_GEX'] = cgex
        dfsumgex.loc[level, 'p_GEX'] = pgex
        dfsumgex.loc[level, 'GEX'] = ngex
    # Obtain zero gamma level by linear interpolation
    if (dfsumgex['GEX'].min() * dfsumgex['GEX'].max()) > 0:
        zerogexlevel = None
    else:
        for p1, p2 in zip(dfsumgex.index[:-1], dfsumgex.index[1:]):
            level1 = dfsumgex.loc[p1, 'GEX']
            level2 = dfsumgex.loc[p2, 'GEX']
            if level1 * level2 < 0:
                zerogexlevel = round((p1 * level2 - p2 * level1) / (level2 - level1), 2)
                break
    # Visualize GEX at different spot price 
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, row_heights=[6.0], specs=[[{"type": "scatter"}]])
    
    fig.add_trace(go.Scatter(x=dfsumgex.index, y=dfsumgex['GEX'], mode='lines+markers', name='spot levels'), 
                  row=1, col=1)
    
    fig.add_trace(go.Scatter(x=[spotprice], y=[ngex0], mode='markers', name='spot price', 
                             marker=dict(size=20, color='#66dc19')), row=1, col=1)
    
    if zerogexlevel != None:
        fig.add_trace(go.Scatter(x=[zerogexlevel], y=[0], mode='markers', name='0-gamma level', 
                                 marker=dict(size=20, color=' #ee195a')), row=1, col=1)
        
    tdate = dfchain['tradedate'].iloc[0].strftime('%Y-%m-%d')
    expiry = (dfchain['expiry'].iloc[0]).strftime('%Y-%m-%d')
    fig.update_layout(title=f'Gamma levels of {asset} of expiry {expiry} on {tdate}', 
                      title_x=0.5, width=1000, height=800)
    fig.show()

    return dfsumgex, zerogexlevel

In [12]:
lowb1 = dfop1[f'{asset0}_cl'].mean() * 0.8
uppb1 = dfop1[f'{asset0}_cl'].mean() * 1.2
dfsumgex1, zerogex1 = get0gamma(dfop1, asset=asset0, spotbound=(lowb1, uppb1))